In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import time 

from sklearn.base import TransformerMixin
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.utils import resample
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,roc_auc_score,classification_report,roc_curve,auc, f1_score

import import_ipynb
from function_for_eda import *

C:\Users\Admin\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\Admin\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


importing Jupyter notebook from function_for_eda.ipynb


# 1. BUREAU_BALANCE.CSV

Bảng bureau_balance chứa các thông tin về các khoản vay của khách hàng bên ngoài Home Credit và được báo cáo bởi Credit Bureau Department
Từ các bước EDA trước đó, chúng ta tiến hành các bước Feature Engineering như sau:
- <b>Bước 1</b>: Ở cột STATUS sẽ không dùng hàm encode chung mà sẽ đi label encoding nó.
- <b>Bước 2</b>: Có thể thấy data chứa dữ liệu TimeSeries, nên ở đây có thể dùng các phương pháp như Exponential Weighted Moving Average (EMA) or Weighted Moving Average(WMA) or SMA để xem biến động các giá trị trung bình trong một khoảng thời gian nhất định.
- <b>Bước 3</b>: Thực hiện các aggregations thông qua SK_ID_BUREAU bằng các phép aggregation như min, max, sum, count,.... Việc lựa chọn các phép aggregation dựa trên domain knowledge cũng như một số solutions tham khảo khác
- <b>Bước 4</b>: Ngoài ra, nhóm 12 cũng thực hiện aggregation dựa trên những bản ghi trong 2 năm gần nhất. Vì 2 năm này sẽ mô tả đúng hơn các hành vi của khách hàng
 

In [2]:
bureau_balance = pd.read_csv('../dseb63_final_project_DP_dataset/dseb63_bureau_balance.csv')
bureau_balance

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C
...,...,...,...
27299920,5041336,-47,X
27299921,5041336,-48,X
27299922,5041336,-49,X
27299923,5041336,-50,X


In [3]:
dict_for_status = { 'C': 0, '0': 1, '1': 2, '2': 3, 'X': 4, '3': 5, '4': 6, '5': 7}
bureau_balance['STATUS'] = bureau_balance['STATUS'].map(dict_for_status)


In [4]:
#weighing the status with the months_balance
#converting months to positive
bureau_balance['MONTHS_BALANCE'] = np.abs(bureau_balance['MONTHS_BALANCE'])
bureau_balance['WEIGHTED_STATUS'] = bureau_balance.STATUS / (bureau_balance.MONTHS_BALANCE + 1)


In [5]:
bureau_balance = bureau_balance.sort_values(by=['SK_ID_BUREAU', 'MONTHS_BALANCE'], ascending=[0, 0])

In [6]:
#we will do exponential weighted average on the encoded status
#this is because if a person had a bad status 2 years ago, it should be given less weightage today
bureau_balance['EXP_WEIGHTED_STATUS'] = bureau_balance.groupby('SK_ID_BUREAU')['WEIGHTED_STATUS'].transform(lambda x: x.ewm(alpha = 0.8).mean())
bureau_balance['EXP_ENCODED_STATUS'] = bureau_balance.groupby('SK_ID_BUREAU')['STATUS'].transform(lambda x: x.ewm(alpha = 0.8).mean())    

In [7]:
bureau_balance

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS,WEIGHTED_STATUS,EXP_WEIGHTED_STATUS,EXP_ENCODED_STATUS
23544064,6842888,61,1,0.016129,1.612903e-02,1.000000e+00
23544063,6842888,60,1,0.016393,1.634937e-02,1.000000e+00
23544062,6842888,59,1,0.016667,1.660526e-02,1.000000e+00
23544061,6842888,58,2,0.033898,3.046187e-02,1.801282e+00
23544060,6842888,57,1,0.017241,1.988209e-02,1.160051e+00
...,...,...,...,...,...,...
26079757,5001709,4,0,0.000000,2.217031e-59,1.934281e-57
26079756,5001709,3,0,0.000000,4.434062e-60,3.868563e-58
26079755,5001709,2,0,0.000000,8.868123e-61,7.737125e-59
26079754,5001709,1,0,0.000000,1.773625e-61,1.547425e-59


In [8]:
aggregations_basic = {
    'MONTHS_BALANCE' : ['mean','max'],
    'STATUS' : ['mean','max','first'],
    'WEIGHTED_STATUS' : ['mean','sum','first'],
    'EXP_ENCODED_STATUS' : ['last'],
    'EXP_WEIGHTED_STATUS' : ['last']}

#aggregating over whole dataset first
aggregated_bureau_balance = bureau_balance.groupby(['SK_ID_BUREAU']).agg(aggregations_basic)
aggregated_bureau_balance.columns = ['_'.join(ele).upper() for ele in aggregated_bureau_balance.columns]



In [9]:
#we will be finding aggregates for each year too
aggregations_for_year = {
    'STATUS' : ['mean','max','last','first'],
    'WEIGHTED_STATUS' : ['mean','max', 'first','last'],
    'EXP_WEIGHTED_STATUS' : ['last'],
    'EXP_ENCODED_STATUS' : ['last']}

#aggregating some of the features separately for latest 2 years
aggregated_bureau_years = pd.DataFrame()
for year in range(2):
    year_group = bureau_balance[bureau_balance['MONTHS_BALANCE'] == year].groupby('SK_ID_BUREAU').agg(aggregations_for_year)
    year_group.columns = ['_'.join(ele).upper() + '_YEAR_' + str(year) for ele in year_group.columns]

    if year == 0:
        aggregated_bureau_years = year_group
    else:
        aggregated_bureau_years = aggregated_bureau_years.merge(year_group, on = 'SK_ID_BUREAU', how = 'outer')

#aggregating for rest of the years
aggregated_bureau_rest_years = bureau_balance[bureau_balance.MONTHS_BALANCE > year].groupby(['SK_ID_BUREAU']).agg(aggregations_for_year)
aggregated_bureau_rest_years.columns = ['_'.join(ele).upper() + '_YEAR_REST' for ele in aggregated_bureau_rest_years.columns]

#merging with rest of the years
aggregated_bureau_years = aggregated_bureau_years.merge(aggregated_bureau_rest_years, on = 'SK_ID_BUREAU', how = 'outer')
aggregated_bureau_balance = aggregated_bureau_balance.merge(aggregated_bureau_years, on = 'SK_ID_BUREAU', how = 'inner')

#filling the missing values by our function fill_nan
aggregated_bureau_balance = fill_nan(aggregated_bureau_balance)


In [10]:
aggregated_bureau_balance

,MONTHS_BALANCE_MEAN,MONTHS_BALANCE_MAX,STATUS_MEAN,STATUS_MAX,STATUS_FIRST,WEIGHTED_STATUS_MEAN,WEIGHTED_STATUS_SUM,WEIGHTED_STATUS_FIRST,EXP_ENCODED_STATUS_LAST,EXP_WEIGHTED_STATUS_LAST,...,STATUS_MEAN_YEAR_REST,STATUS_MAX_YEAR_REST,STATUS_LAST_YEAR_REST,STATUS_FIRST_YEAR_REST,WEIGHTED_STATUS_MEAN_YEAR_REST,WEIGHTED_STATUS_MAX_YEAR_REST,WEIGHTED_STATUS_FIRST_YEAR_REST,WEIGHTED_STATUS_LAST_YEAR_REST,EXP_WEIGHTED_STATUS_LAST_YEAR_REST,EXP_ENCODED_STATUS_LAST_YEAR_REST
SK_ID_BUREAU,,,,,,,,,,,,,,,,,,,,,
5001709,48.0,96,0.453608,4,4,0.004936,0.478827,0.041237,3.094850e-60,3.547249e-62,...,0.463158,4.0,0.0,4.0,0.005040,0.045977,0.041237,0.000000,8.868123e-61,7.737125e-59
5001710,41.0,82,1.506024,4,4,0.022689,1.883185,0.048193,4.166370e-34,8.419081e-36,...,1.543210,4.0,0.0,4.0,0.023249,0.080000,0.048193,0.000000,2.104770e-34,1.041593e-32
5001711,1.5,3,1.750000,4,1,1.270833,5.083333,0.250000,3.403846e+00,3.297543e+00,...,1.000000,1.0,1.0,1.0,0.291667,0.333333,0.250000,0.333333,3.194444e-01,1.000000e+00
5001712,9.0,18,0.526316,1,1,0.037830,0.718771,0.052632,5.119999e-07,5.008223e-08,...,0.588235,1.0,0.0,1.0,0.042281,0.100000,0.052632,0.000000,1.252056e-06,1.280000e-05
5001713,10.5,21,4.000000,4,4,0.671057,14.763253,0.181818,4.000000e+00,3.570297e+00,...,4.000000,4.0,4.0,4.0,0.438163,1.333333,0.181818,1.333333,1.257421e+00,4.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6842884,23.5,47,1.770833,4,4,0.058626,2.814045,0.083333,4.194304e-14,1.975070e-15,...,1.847826,4.0,0.0,4.0,0.061175,0.190476,0.083333,0.000000,4.937674e-14,1.048576e-12
6842885,11.5,23,4.000000,7,1,0.933134,22.395222,0.041667,7.000000e+00,6.248019e+00,...,3.727273,7.0,7.0,1.0,0.540692,2.333333,0.041667,2.333333,2.200486e+00,6.999999e+00
6842886,16.0,32,0.242424,1,1,0.008268,0.272840,0.030303,3.355435e-18,1.278811e-19,...,0.258065,1.0,0.0,1.0,0.008801,0.038462,0.030303,0.000000,3.197028e-18,8.388587e-17


# 2. Bureau.csv

Cũng giống bảng bureau_balance, bureau chứa các thông tin về các khoản vay của khách hàng bên ngoài Home Credit và được báo cáo bởi Credit Bureau Department
Từ các bước EDA trước đó, chúng ta tiến hành các bước Feature Engineering như sau:
- <b> Bước 1 </b>: Chúng ta merge bảng bureau_balance đã engineering thông qua SK_ID_BURREAU
- <b> Bước 2 </b>: Ở EDA chúng ta phát hiện các giá trị bất thường ở các biến 'DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT', 'DAYS_CREDIT_UPDATE', tiến hành xử lý những giá trị này
- <b> Bước 3 </b>: tạo các feature mới bằng các phép, cộng, trừ, nhân, chia từ các feature gốc bằng domain knowledge, quá trình eda, cũng như một số solutions tham khảo khác
- <b> Bước 4 </b>: Encoding các biến categorical
- <b> Bước 5 </b>: Thực hiện các aggregations thông qua SK_ID_CURR bằng các phép aggregation như min, max, sum, count,....Như Trong qua trình EDA đã phát hiện, chúng tôi tập trung aggreagated qua 2 most category in CREDIT_TYPE là Active and Closed, sau đó aggregate qua những categories còn lại. Sau đó aggregate cho toàn bộ data.  Việc lựa chọn các phép aggregation dựa trên domain knowledge, quá trình eda, cũng như một số solutions tham khảo khác 

In [11]:
bureau = pd.read_csv('../dseb63_final_project_DP_dataset/dseb63_bureau.csv')
bureau

,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,SK_ID_CURR
0,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,254629
1,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN,254629
2,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN,254629
3,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN,254629
4,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN,254629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465320,5052629,Closed,currency 1,-532,0,45.0,-227.0,NaN,0,450000.0,0.0,0.0,0.0,Consumer credit,-31,NaN,66820
1465321,5054382,Closed,currency 1,-381,0,-199.0,-199.0,0.0,0,19800.0,0.0,0.0,0.0,Consumer credit,-199,0.0,166221
1465322,5043236,Active,currency 1,-1441,0,2197.0,NaN,NaN,0,1800000.0,1076809.5,0.0,0.0,Mortgage,-2,NaN,118290
1465323,5053303,Closed,currency 1,-375,0,-41.0,-19.0,NaN,0,175054.5,NaN,NaN,0.0,Consumer credit,-4,17140.5,95296


In [12]:
bureau.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465325 entries, 0 to 1465324
Data columns (total 17 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   SK_ID_BUREAU            1465325 non-null  int64  
 1   CREDIT_ACTIVE           1465325 non-null  object 
 2   CREDIT_CURRENCY         1465325 non-null  object 
 3   DAYS_CREDIT             1465325 non-null  int64  
 4   CREDIT_DAY_OVERDUE      1465325 non-null  int64  
 5   DAYS_CREDIT_ENDDATE     1376227 non-null  float64
 6   DAYS_ENDDATE_FACT       920652 non-null   float64
 7   AMT_CREDIT_MAX_OVERDUE  516780 non-null   float64
 8   CNT_CREDIT_PROLONG      1465325 non-null  int64  
 9   AMT_CREDIT_SUM          1465322 non-null  float64
 10  AMT_CREDIT_SUM_DEBT     1242231 non-null  float64
 11  AMT_CREDIT_SUM_LIMIT    975655 non-null   float64
 12  AMT_CREDIT_SUM_OVERDUE  1465325 non-null  float64
 13  CREDIT_TYPE             1465325 non-null  object 
 14  DA

In [13]:
bureau_merged = bureau.merge(aggregated_bureau_balance, on = 'SK_ID_BUREAU', how = 'left')
bureau_merged

,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,...,STATUS_MEAN_YEAR_REST,STATUS_MAX_YEAR_REST,STATUS_LAST_YEAR_REST,STATUS_FIRST_YEAR_REST,WEIGHTED_STATUS_MEAN_YEAR_REST,WEIGHTED_STATUS_MAX_YEAR_REST,WEIGHTED_STATUS_FIRST_YEAR_REST,WEIGHTED_STATUS_LAST_YEAR_REST,EXP_WEIGHTED_STATUS_LAST_YEAR_REST,EXP_ENCODED_STATUS_LAST_YEAR_REST
0,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465320,5052629,Closed,currency 1,-532,0,45.0,-227.0,NaN,0,450000.0,...,0.375000,1.0,0.0,1.0,0.024494,0.076923,0.055556,0.000000,7.740404e-09,1.023934e-07
1465321,5054382,Closed,currency 1,-381,0,-199.0,-199.0,0.0,0,19800.0,...,1.000000,1.0,1.0,1.0,0.152739,0.333333,0.076923,0.333333,3.143551e-01,1.000000e+00
1465322,5043236,Active,currency 1,-1441,0,2197.0,NaN,NaN,0,1800000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1465323,5053303,Closed,currency 1,-375,0,-41.0,-19.0,NaN,0,175054.5,...,1.000000,1.0,1.0,1.0,0.152739,0.333333,0.076923,0.333333,3.143551e-01,1.000000e+00


In [14]:
bureau_merged['DAYS_CREDIT_ENDDATE'][bureau_merged['DAYS_CREDIT_ENDDATE'] > -50*365] = np.nan
bureau_merged['DAYS_ENDDATE_FACT'][bureau_merged['DAYS_ENDDATE_FACT'] > -50*365] = np.nan
#there is also a feature which tells about the number of days ago the Credit Report Came
bureau_merged['DAYS_CREDIT_UPDATE'][bureau_merged['DAYS_CREDIT_UPDATE'] > -50*365] = np.nan

#engineering some features based on domain knowledge
bureau_merged['CREDIT_DURATION'] = np.abs(bureau_merged['DAYS_CREDIT'] - bureau_merged['DAYS_CREDIT_ENDDATE'])
bureau_merged['FLAG_OVERDUE_RECENT'] = [0 if ele == 0 else 1 for ele in bureau_merged['CREDIT_DAY_OVERDUE']]
bureau_merged['MAX_AMT_OVERDUE_DURATION_RATIO'] = bureau_merged['AMT_CREDIT_MAX_OVERDUE'] / (bureau_merged['CREDIT_DURATION'] + 0.00001)
bureau_merged['CURRENT_AMT_OVERDUE_DURATION_RATIO'] = bureau_merged['AMT_CREDIT_SUM_OVERDUE'] / (bureau_merged['CREDIT_DURATION'] + 0.00001)
bureau_merged['AMT_OVERDUE_DURATION_LEFT_RATIO'] = bureau_merged['AMT_CREDIT_SUM_OVERDUE'] / (bureau_merged['DAYS_CREDIT_ENDDATE'] + 0.00001)
bureau_merged['CNT_PROLONGED_MAX_OVERDUE_MUL'] = bureau_merged['CNT_CREDIT_PROLONG'] * bureau_merged['AMT_CREDIT_MAX_OVERDUE']
bureau_merged['CNT_PROLONGED_DURATION_RATIO'] = bureau_merged['CNT_CREDIT_PROLONG'] / (bureau_merged['CREDIT_DURATION'] + 0.00001)
bureau_merged['CURRENT_DEBT_TO_CREDIT_RATIO'] = bureau_merged['AMT_CREDIT_SUM_DEBT'] / (bureau_merged['AMT_CREDIT_SUM'] + 0.00001)
bureau_merged['CURRENT_CREDIT_DEBT_DIFF'] = bureau_merged['AMT_CREDIT_SUM'] - bureau_merged['AMT_CREDIT_SUM_DEBT']
bureau_merged['AMT_ANNUITY_CREDIT_RATIO'] = bureau_merged['AMT_ANNUITY'] / (bureau_merged['AMT_CREDIT_SUM'] + 0.00001)
bureau_merged['CREDIT_ENDDATE_UPDATE_DIFF'] = np.abs(bureau_merged['DAYS_CREDIT_UPDATE'] - bureau_merged['DAYS_CREDIT_ENDDATE'])


In [15]:
aggregations_CREDIT_ACTIVE = {
                        'DAYS_CREDIT' : ['mean','min','max','last'],
                        'CREDIT_DAY_OVERDUE' : ['mean','max'],
                        'DAYS_CREDIT_ENDDATE' : ['mean','max'],
                        'DAYS_ENDDATE_FACT' : ['mean','min'],
                        'AMT_CREDIT_MAX_OVERDUE': ['max','sum'],
                        'CNT_CREDIT_PROLONG': ['max','sum'],
                        'AMT_CREDIT_SUM' : ['sum','max'],
                        'AMT_CREDIT_SUM_DEBT': ['sum'],
                        'AMT_CREDIT_SUM_LIMIT': ['max','sum'],
                        'AMT_CREDIT_SUM_OVERDUE': ['max','sum'],
                        'DAYS_CREDIT_UPDATE' : ['mean','min'],
                        'AMT_ANNUITY' : ['mean','sum','max'],
                        'CREDIT_DURATION' : ['max','mean'],
                        'FLAG_OVERDUE_RECENT': ['sum'],
                        'MAX_AMT_OVERDUE_DURATION_RATIO' : ['max','sum'],
                        'CURRENT_AMT_OVERDUE_DURATION_RATIO' : ['max','sum'],
                        'AMT_OVERDUE_DURATION_LEFT_RATIO' : ['max', 'mean'],
                        'CNT_PROLONGED_MAX_OVERDUE_MUL' : ['mean','max'],
                        'CNT_PROLONGED_DURATION_RATIO' : ['mean', 'max'],
                        'CURRENT_DEBT_TO_CREDIT_RATIO' : ['mean', 'min'],
                        'CURRENT_CREDIT_DEBT_DIFF' : ['mean','min'],
                        'AMT_ANNUITY_CREDIT_RATIO' : ['mean','max','min'],
                        'CREDIT_ENDDATE_UPDATE_DIFF' : ['max','min'],
                        'STATUS_MEAN' : ['mean', 'max'],
                        'WEIGHTED_STATUS_MEAN' : ['mean', 'max']
                         }


In [17]:
#we saw from EDA that the two most common type of CREDIT ACTIVE were 'Closed' and 'Active'.
#So we will aggregate them two separately and the remaining categories separately.
categories_to_aggregate_on = ['Closed','Active']
bureau_merged_aggregated_credit = pd.DataFrame()
for i, status in enumerate(categories_to_aggregate_on):
    group = bureau_merged[bureau_merged['CREDIT_ACTIVE'] == status].groupby('SK_ID_CURR').agg(aggregations_CREDIT_ACTIVE)
    group.columns = ['_'.join(ele).upper() + '_CREDITACTIVE_' + status.upper() for ele in group.columns]

    if i==0:
        bureau_merged_aggregated_credit = group
    else:
        bureau_merged_aggregated_credit = bureau_merged_aggregated_credit.merge(group, on = 'SK_ID_CURR', how = 'outer')
 #aggregating for remaining categories
bureau_merged_aggregated_credit_rest = bureau_merged[(bureau_merged['CREDIT_ACTIVE'] != 'Active') & 
                                                     (bureau_merged['CREDIT_ACTIVE'] != 'Closed')].groupby('SK_ID_CURR').agg(aggregations_CREDIT_ACTIVE)
bureau_merged_aggregated_credit_rest.columns = ['_'.join(ele).upper() + 'CREDIT_ACTIVE_REST' for ele in bureau_merged_aggregated_credit_rest.columns]

#merging with other categories
bureau_merged_aggregated_credit = bureau_merged_aggregated_credit.merge(bureau_merged_aggregated_credit_rest, on = 'SK_ID_CURR', how = 'outer')

#encoding categorical features
bureau_merged = encode(bureau_merged)

#aggregating the bureau_merged over all the columns
bureau_merged_aggregated = bureau_merged.drop('SK_ID_BUREAU', axis = 1).groupby('SK_ID_CURR').agg('mean')
bureau_merged_aggregated.columns = [ele + '_MEAN_OVERALL' for ele in bureau_merged_aggregated.columns]

#merging it with aggregates over categories
bureau_merged_aggregated = bureau_merged_aggregated.merge(bureau_merged_aggregated_credit, on = 'SK_ID_CURR', how = 'outer')


In [18]:
bureau_merged_aggregated.reset_index(inplace = True)

In [19]:
bureau_merged_aggregated

,SK_ID_CURR,DAYS_CREDIT_MEAN_OVERALL,CREDIT_DAY_OVERDUE_MEAN_OVERALL,DAYS_CREDIT_ENDDATE_MEAN_OVERALL,DAYS_ENDDATE_FACT_MEAN_OVERALL,AMT_CREDIT_MAX_OVERDUE_MEAN_OVERALL,CNT_CREDIT_PROLONG_MEAN_OVERALL,AMT_CREDIT_SUM_MEAN_OVERALL,AMT_CREDIT_SUM_DEBT_MEAN_OVERALL,AMT_CREDIT_SUM_LIMIT_MEAN_OVERALL,...,CURRENT_CREDIT_DEBT_DIFF_MINCREDIT_ACTIVE_REST,AMT_ANNUITY_CREDIT_RATIO_MEANCREDIT_ACTIVE_REST,AMT_ANNUITY_CREDIT_RATIO_MAXCREDIT_ACTIVE_REST,AMT_ANNUITY_CREDIT_RATIO_MINCREDIT_ACTIVE_REST,CREDIT_ENDDATE_UPDATE_DIFF_MAXCREDIT_ACTIVE_REST,CREDIT_ENDDATE_UPDATE_DIFF_MINCREDIT_ACTIVE_REST,STATUS_MEAN_MEANCREDIT_ACTIVE_REST,STATUS_MEAN_MAXCREDIT_ACTIVE_REST,WEIGHTED_STATUS_MEAN_MEANCREDIT_ACTIVE_REST,WEIGHTED_STATUS_MEAN_MAXCREDIT_ACTIVE_REST
0,0,-63.000000,0.0,NaN,NaN,NaN,0.0,8.551350e+04,77566.500000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,-2348.000000,0.0,NaN,NaN,11666.385,0.0,2.835000e+04,0.000000,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,-810.333333,0.0,NaN,NaN,3771.990,0.0,4.239018e+04,16069.500000,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,-1131.428571,0.0,NaN,NaN,0.000,0.0,6.913607e+04,28395.642857,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,-773.333333,0.0,NaN,NaN,NaN,0.0,4.704240e+05,209084.250000,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263486,307504,-1149.750000,0.0,NaN,NaN,0.000,0.0,2.165383e+05,29941.875000,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
263487,307505,-461.500000,0.0,NaN,NaN,0.000,0.0,1.809000e+06,0.000000,540000.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
263488,307506,-1390.500000,0.0,NaN,NaN,NaN,0.0,6.481575e+04,0.000000,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
263489,307508,-765.428571,0.0,NaN,NaN,0.000,0.0,7.866153e+05,41958.000000,300.7875,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
bureau_merged_aggregated.to_csv('bureau_balance_final.csv', index = False)
print('done')

done
